# Lab 2 - Enter the Model Context Protocol!

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
load_dotenv(override=True)
MODEL_NAME = "gpt-4.1-mini"

# First - around the table:

### What is an MCP Server - how do you think of it?

### How would you explain why it's exciting to a non-technical person?


In [2]:
# An MCP Server is defined by Parameters

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
    tools = await fetch.session.list_tools()

tools.tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [8]:
# Fix PATH to include Homebrew node/npx
import os
os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ.get("PATH", "")


In [9]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
    file_tools = await filesystem.session.list_tools()

file_tools.tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

In [10]:
instructions = "You use your tools to navigate the web and write summaries to a file"

input = "Bring up a browser, visit 3 different news sites, click on at least 1 story on each site, and write a summary of the news in markdown to a file news.md in the sandbox directory"

In [ ]:
with trace("News"):
    async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[fetch, filesystem])
            result = await Runner.run(agent, input)
            print(result.final_output)

I have written a summary of notable news stories from BBC, CNN, and The New York Times into a markdown file named news.md in the sandbox directory. If you want, I can show you the contents of the file or assist you with anything else.


# And now.. a hands on moment

### Everyone write a version of this that uses the MCP Server from Playwright instead of from Fetch

Using this MCP Server from Microsoft:

https://mcp.so/server/playwright-mcp/microsoft

But NOT running it in headless mode, so you get to see the browser

In [14]:
# Simple Playwright version - just like fetch!
playwright_params = {"command": "npx", "args": ["-y", "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as playwright:
    async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as filesystem:
        agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[playwright, filesystem])
        result = await Runner.run(agent, input, max_turns=30)  # Browser automation takes more steps!
        print(result.final_output)


I have completed visiting three news sites, explored a story from each, and written a summary in markdown format to the file news.md in the sandbox directory. You can access the consolidated news summary there. If you need anything else, please let me know!


### And check out the traces

https://platform.openai.com/traces